October 29th, 2024  
PG-bot  
Code Guru: sam-i-am


## Outline

* In pg-bot.py
    * get prompt from user
    * vectorize prompt
    * query Neon
    * Format for App output (retrieve formatted)

In [2]:
# this is needed to import the module pg-bot
import sys
sys.path.append('/home/sng/pg-bot/pg-bot') 

from config import get_client, EMBEDDING_MODEL
import pandas as pd

%load_ext autoreload
%autoreload 2

### Initialize the  OpenAIclient

In [8]:
client = get_client()
try:
    # Make a simple test API call
    response = client.models.list()
    print("OpenAI client is connected!", response)
except Exception as e:
    print(f"Error connecting to OpenAI: {e}")

OpenAI client is connected! SyncPage[Model](data=[Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'), Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo-0125

In [15]:
# get vector from openai
import oai_utils as oai
import neondb as neon

vector = oai.vectorize_data_with_openai('Tell me about UV Lamp Installation/Removal Instructions')
vector



<connection object at 0x7f938845d580; dsn: 'user=neondb_owner password=xxx dbname=neondb host=ep-wandering-dawn-a565vw7x.us-east-2.aws.neon.tech sslmode=require', closed: 0>

In [ ]:
# query neon
conn = neon.connect_db()
conn


In [16]:
neon.list_tables()

['chunk_embeds',
 'chunk_kwds',
 'element_chunk_junction',
 'raw_elements',
 'tool_table']

In [17]:
neon.list_columns('chunk_embeds')

[('chunk_id', 'text'),
 ('parent_type', 'text'),
 ('parent_content', 'text'),
 ('name_of_tool', 'text'),
 ('content', 'text'),
 ('embedding', 'USER-DEFINED')]

In [19]:
retrieved_chunks = neon.get_top_k_similar_docs(vector, 10)
retrieved_chunks


[('mc2scgxelv2e4pph41upli2m',
  'header',
  'INSTALLATION (continued)',
  'Ultron Systems Model UH102/102-12 UV Curing System',
  'INSTALLATION (continued)\nUV Lamp Installation/Removal Instructions'),
 ('z8ukh96mym8mt80xip5ienxz',
  'narrative_text',
  'IMPORTANT NOTE!!!: UV Lamp installation and removal should be done with the power cord unplugged to insure the safety of the operator.',
  'Ultron Systems Model UH102/102-12 UV Curing System',
  'IMPORTANT NOTE!!!: UV Lamp installation and removal should be done with the power cord unplugged to insure the safety of the operator.\n\na) Raise the Door of the unit until it rests against the stops.\n\nb) Loosen the two 10/32” hex cap screws which secures the base plate to the unit (Door Frame Lock Screws) located on the inside front edge of the Door Frame on the Frame Lock (Figure 2).\n\nc) Remove the Frame Lock by pulling it towards the center of the unit. (Figure 2).\n\nd) Raise the Door Frame (hinged at the rear), and push it up against

In [20]:
df = pd.DataFrame(retrieved_chunks, columns=['chunk_id', 'parent_type', 
                                          'parent_content', 'name_of_tool', 
                                          'content'])

In [21]:
df

,chunk_id,parent_type,parent_content,name_of_tool,content
0,mc2scgxelv2e4pph41upli2m,header,INSTALLATION (continued),Ultron Systems Model UH102/102-12 UV Curing Sy...,INSTALLATION (continued)\nUV Lamp Installation...
1,z8ukh96mym8mt80xip5ienxz,narrative_text,IMPORTANT NOTE!!!: UV Lamp installation and re...,Ultron Systems Model UH102/102-12 UV Curing Sy...,IMPORTANT NOTE!!!: UV Lamp installation and re...
2,up7o0pghxvjc8zphf2phpcax,header,INSTALLATION (continued),Ultron Systems Model UH102/102-12 UV Curing Sy...,INSTALLATION (continued)\nUV Lamp Installation...
3,k22fpmx7ql8uwa08jrjcoabv,header,UV LAMP MEASUREMENT PROCEDURE,Ultron Systems Model UH102/102-12 UV Curing Sy...,UV LAMP MEASUREMENT PROCEDURE\na) Make certain...
4,zk017xheg7tuwnx0mgmd3e20,header,INSTALLATION,Ultron Systems Model UH102/102-12 UV Curing Sy...,INSTALLATION\nThe Model UH102 Manual UV Curing...
5,o6a2kd5j3t58ha1j5u92jl7e,header,UV LAMP MEASUREMENT PROCEDURE,Ultron Systems Model UH102/102-12 UV Curing Sy...,UV LAMP MEASUREMENT PROCEDURE\nUltron Systems ...
6,vbofhr5257pwro6z976pgdrn,image,Diagram labeled 'Figure 4' showing the UH102 l...,Ultron Systems Model UH102/102-12 UV Curing Sy...,Diagram labeled 'Figure 4' showing the UH102 l...
7,ao3qiw8s0ektny9uvmwza7o0,header,OPERATION,Ultron Systems Model UH102/102-12 UV Curing Sy...,OPERATION\na) Depress the POWER button to turn...
8,xwy6rskr2rs0dirywcuz8757,header,UV LAMP MEASUREMENT PROCEDURE (continued),Ultron Systems Model UH102/102-12 UV Curing Sy...,UV LAMP MEASUREMENT PROCEDURE (continued)\nj) ...
9,f6en6onlvaybhpdxahj0fiky,header,CONTROLS AND ADJUSTMENTS (continued),Ultron Systems Model UH102/102-12 UV Curing Sy...,CONTROLS AND ADJUSTMENTS (continued)\nNUMERIC ...


In [23]:
tools = neon.get_tool_names()
tools

['Ultron Systems Model UH102/102-12 UV Curing System']

## Query Neon
1. Get user prompt
2. Send prompt to openai embeddings api and extract the vector
3. Send the vector to Neon to find nearest neighbors
4. Get the retrieved chunks

### Fill in your prompt info below 

In [ ]:
prompt = 'type your questions here'
k = 10 # number of retrievals
tenant_name = 'your_tenant_name'
weaviate_class = 'your class_name'

In [ ]:
retrieved_texts = weav.query_weaviate(prompt, k, weaviate_class, tenant_name)
retrieved_texts